## Job Generation

In [ ]:
import numpy as np
import scipy
import matplotlib.pyplot as plt
import csv

## Constant Parameters

In [ ]:
# number of jobs
n = 10000
number_of_queues = 4
lmbda = 0.1
mu = 0.2
time_slice = 4.5
filePath = "jobs.csv"

In [ ]:

jobsArray = []

class Job:
    def __init__(self, arrival_time, service_time, priority):
        self.arrival_time = arrival_time
        self.service_time = service_time
        self.priority = priority
        self.sojourn_time = 0
        self.latency_due_to_preemption = 0
        self.waiting_time = 0
        self.pre_waiting_time = 0
        self.service_start_time = None
        self.service_end_time = None
        self.last_service_time = None
        self.isJobDone = False
        self.service_left = service_time
        self.num_preemptions = 0

def load_jobs_from_csv(file_path):
    jobs = []
    with open(file_path, 'r') as csv_file:
        csv_reader = csv.reader(csv_file)
        next(csv_reader)  # Skip header if present
        for row in csv_reader:
            arrival_time, service_time, priority = map(float, row)
            job = Job(arrival_time, service_time, priority)
            jobs.append(job)
    return jobs        

def generate_exponential_xi(n, lmbda):
    y = np.random.uniform(0, 1, n)
    x = [-np.log(1 - y[i]) / lmbda for i in range(len(y))]
    return x

def generate_si_sigma_xi(n):
    n_sorted = sorted(n)
    si_sigma_xi = []
    for i in range(len(n_sorted)):
        si_sigma_xi.append(sum(n_sorted[:i+1]))
    return si_sigma_xi

def generate_sizes(si_sigma_xi, mu):
    sizes = generate_exponential_xi(len(si_sigma_xi), mu)
    jobs = []
    for i in range(len(si_sigma_xi)):
        jobs.append(Job(si_sigma_xi[i], sizes[i], None))
    return jobs

def assign_priority(jobsArray, number_of_queues, average_service_time):
    bin_size = 2 * average_service_time / number_of_queues
    for job in jobsArray:
        priority = int(job.service_time / bin_size)
        if priority >= number_of_queues:
            priority = number_of_queues - 1
        job.priority = priority
    return jobsArray

def areJobsLeft(jobsArray):
    for job in jobsArray:
        if job.isJobDone == False:   
            return True
    return False

def getHighestPriorityJob(jobsArray,global_clock):
    index = -1
    priority = 100000
    for i, job in enumerate(jobsArray):
        if job.priority <= priority and global_clock >= job.arrival_time and job.isJobDone == False:
            priority = job.priority
            index = i 
            
    return index

def getNextJobArrival(jobsArray):
    for i, job in enumerate(jobsArray):
        if job.isJobDone == False:
            return i
    return None  # Return None if no more jobs are left

def simulate_queue(jobsArray, numJobs, time_slice):
    global_clock = 0
    numJobsdone = 0
    time_slice = time_slice
    last_job_index = -1
    while numJobsdone != numJobs:
        highest_priority_job_index = getHighestPriorityJob(jobsArray, global_clock)

        if highest_priority_job_index == -1:
            index = getNextJobArrival(jobsArray)
            if index is None:
                break  # No more jobs left, exit the loop
            global_clock = jobsArray[index].arrival_time
            highest_priority_job_index = index

        currJob = jobsArray[highest_priority_job_index]
        if currJob.service_start_time is None:
            jobsArray[highest_priority_job_index].service_start_time = global_clock
            jobsArray[highest_priority_job_index].pre_waiting_time = global_clock - currJob.arrival_time
        else:
            jobsArray[highest_priority_job_index].latency_due_to_preemption += global_clock - jobsArray[highest_priority_job_index].last_service_time
            if last_job_index != highest_priority_job_index:
                jobsArray[highest_priority_job_index].num_preemptions += 1

        if currJob.service_left > time_slice:
            global_clock += time_slice
            jobsArray[highest_priority_job_index].service_left -= time_slice
        else:
            global_clock += currJob.service_left
            jobsArray[highest_priority_job_index].isJobDone = True
            jobsArray[highest_priority_job_index].service_left = 0
            jobsArray[highest_priority_job_index].service_end_time = global_clock
            jobsArray[highest_priority_job_index].waiting_time = jobsArray[highest_priority_job_index].pre_waiting_time + jobsArray[highest_priority_job_index].latency_due_to_preemption
            jobsArray[highest_priority_job_index].sojourn_time = jobsArray[highest_priority_job_index].waiting_time + jobsArray[highest_priority_job_index].service_time
            numJobsdone += 1

        last_job_index = highest_priority_job_index
        jobsArray[highest_priority_job_index].last_service_time = global_clock


## Queuing parameters

In [ ]:

xi = generate_exponential_xi(n, lmbda)
si_sigma_xi = generate_si_sigma_xi(xi)
jobsArray = generate_sizes(si_sigma_xi, mu)
jobsArray = assign_priority(jobsArray, number_of_queues, 1/mu)    
# Count number of jobs in each queue
count = [0] * number_of_queues
for job in jobsArray:
    count[job.priority] += 1
print("Counts per priority:", count)

jobsArray = load_jobs_from_csv(filePath)

# Simulate the queue with the given arrival times and sizes using sjf
simulate_queue(jobsArray,n,time_slice)


In [ ]:

print("Jobs:")
for job in jobsArray:
    print("Arrival Time:", job.arrival_time, "Service Time:", job.service_time, "Priority:", job.priority, "Waiting Time:", job.pre_waiting_time)


## Average Service Time for simulation

In [ ]:

def getAverageServiceTime(jobsArray):
    totserviceTime = 0
    for job in jobsArray:
        totserviceTime += job.service_time
    avg_time = totserviceTime/len(jobsArray)
    return avg_time    

def getAverageWaitingTime(jobsArray):
    totWaitingTime = 0
    for job in jobsArray:
        totWaitingTime += job.pre_waiting_time+job.latency_due_to_preemption
    avg_time = totWaitingTime/len(jobsArray)
    return avg_time  

def getAverageSojournTime(jobsArray):
    totSojournTime = 0
    for job in jobsArray:
        totSojournTime += job.sojourn_time
    avg_time = totSojournTime/len(jobsArray)
    return avg_time 

print(getAverageServiceTime(jobsArray))
print(getAverageWaitingTime(jobsArray))
print(getAverageSojournTime(jobsArray))


## Response Time Vs Arrivals

In [ ]:
waitingTimesArray = []

for i, job in enumerate(jobsArray):
    waitingTimesArray.append((job.arrival_time,job.pre_waiting_time,job.priority))
    
print(waitingTimesArray)

priority_data = {}
for arrival_time, pre_waiting_time, priority in waitingTimesArray:
    if priority not in priority_data:
        priority_data[priority] = {'arrival_time': [], 'pre_waiting_time': []}
    priority_data[priority]['arrival_time'].append(arrival_time)
    priority_data[priority]['pre_waiting_time'].append(pre_waiting_time)

for priority in range(4):
    if priority in priority_data:
        data = priority_data[priority]
        plt.plot(data['arrival_time'], data['pre_waiting_time'], label=f'Priority {priority}')

plt.xlabel('Job Arrival')
plt.ylabel('Response Time')
# plt.ylim(0, 100)
plt.xlim(0, 100)
plt.title('Response Time vs Arrival')
plt.legend()
plt.show()

## Waiting Times Vs Arrival

In [ ]:
waitingTimesArray = []

for i, job in enumerate(jobsArray):
    waitingTimesArray.append((job.arrival_time,job.pre_waiting_time + job.latency_due_to_preemption ,job.priority))
    
print(waitingTimesArray)

priority_data = {}
for arrival_time, total_waiting_time, priority in waitingTimesArray:
    if priority not in priority_data:
        priority_data[priority] = {'arrival_time': [], 'total_waiting_time': []}
    priority_data[priority]['arrival_time'].append(arrival_time)
    priority_data[priority]['total_waiting_time'].append(total_waiting_time)

for priority in range(4):
    if priority in priority_data:
        data = priority_data[priority]
        plt.plot(data['arrival_time'], data['total_waiting_time'], label=f'Priority {priority}')

plt.xlabel('Job Arrival')
plt.ylabel('Total Waiting Time')
plt.ylim(0, 10000)
plt.xlim(0, 100)
plt.title('Total Waiting Time vs Arrival')
plt.legend()
plt.show()

## Latency Times Vs Arrival

In [ ]:
waitingTimesArray = []

for i, job in enumerate(jobsArray):
    waitingTimesArray.append((job.arrival_time,job.latency_due_to_preemption ,job.priority))
    
print(waitingTimesArray)

priority_data = {}
for arrival_time, latency_due_to_preemption, priority in waitingTimesArray:
    if priority not in priority_data:
        priority_data[priority] = {'arrival_time': [], 'latency_due_to_preemption': []}
    priority_data[priority]['arrival_time'].append(arrival_time)
    priority_data[priority]['latency_due_to_preemption'].append(latency_due_to_preemption)

for priority in range(4):
    if priority in priority_data:
        data = priority_data[priority]
        plt.plot(data['arrival_time'], data['latency_due_to_preemption'], label=f'Priority {priority}')



plt.xlabel('Job Arrival')
plt.ylabel('Latency Time')
plt.ylim(0, 200)
plt.xlim(0, 100)
plt.title('Latency Time vs Arrival')
plt.legend()
plt.show()

## Number of Preemptions Vs Priority

In [ ]:
#number of preemptions vs priority

preemptions_data = []

for i, job in enumerate(jobsArray):
    preemptions_data.append((job.num_preemptions,job.priority))

priority_data = {}

for num_preemptions, priority in preemptions_data:
    if priority not in priority_data:
        priority_data[priority] = {'num_preemptions': []}
    priority_data[priority]['num_preemptions'].append(num_preemptions)

for priority in range(4):
    if priority in priority_data:
        data = priority_data[priority]
        plt.plot(data['num_preemptions'], label=f'Priority {priority}')

plt.xlabel('Job Arrival')
plt.ylabel('Number of Preemptions')
plt.ylim(0, 4)
plt.xlim(0, 100)
plt.title('Number of Preemptions vs Priority')
plt.legend()
plt.show()

In [ ]:
def plot_time_for_each_job_barh(jobsArray):
    arrival_times = []
    duration = []
    index = []
    count = 0
    for i in jobsArray:
        arrival_times.append(i.arrival_time)
        duration.append(i.service_time + i.waiting_time)
        index.append(count)
        count += 1
    fig, ax = plt.subplots(figsize=(10, 8))
    ax.barh(index, duration, left=arrival_times, color='skyblue', edgecolor='grey')
    ax.set_xlabel('Time')
    ax.set_title('Arrival and Departure Times for Pre-Emptive')
    plt.grid(True)
    plt.show()

def number_of_jobs_in_system_at_any_time(jobsArray):
    arrival_times = []
    departure = []
    timestamps=[]
    counts = []
    unit_time = 0.25
    time = 0
    for i in jobsArray:
        arrival_times.append(i.arrival_time)
        departure.append(i.arrival_time + i.service_time + i.waiting_time)
    while(time<max(departure)):
        timestamps.append(time)
        arrivals = 0
        departures = 0
        for atime in arrival_times:
            if atime<time:
                arrivals+=1
        for dtime in departure:
            if dtime<time:
                departures+=1  
        counts.append(arrivals-departures)  
        time += unit_time
    plt.plot(timestamps,counts)
    plt.xlabel("Time")
    plt.ylabel("Number of jobs in system")
    plt.show()

def which_job_is_being_served(jobsArray):
    timestamps = []
    job_served = []
    colors = []
    time = 0
    unit_time = 0.25
    max_departure = 0
    for job in jobsArray:
        job.first_service = job.arrival_time + job.waiting_time
        job.departure = job.arrival_time+job.waiting_time+job.service_time
        max_departure = max(job.departure,max_departure)     
    while time<max_departure:
        timestamps.append(time)
        flag = 0
        for job in jobsArray:
            if job.first_service<=time and job.departure>time:
                job_served.append(1)
                colors.append(job.color)
                flag = 1
                break
        if flag==0:
            job_served.append(0)
            colors.append("white")
        time += unit_time
    print(colors)
    plt.bar(timestamps,job_served,color=colors)
    plt.xlabel("Time")
    plt.ylabel("Type of job")
    plt.ylim(0,10)
    plt.title("Identity of Jobs with time")
    # plt.savefig("Pre_emptive/arrival_departure.png")
    plt.show()
    
plot_time_for_each_job_barh(jobsArray)
number_of_jobs_in_system_at_any_time(jobsArray)
# which_job_is_being_served(jobsArray)

## Average Number of Pre emptions Vs Priority

In [ ]:
total_jobs_per_priority = [0] * number_of_queues
avg_premptions = [0] * number_of_queues
num_premptions = [0] * number_of_queues

for job in jobsArray:
    total_jobs_per_priority[int(job.priority)] += 1
    num_premptions[int(job.priority)] += job.num_preemptions

for i in range(number_of_queues):
    if total_jobs_per_priority[i] != 0:
        avg_premptions[i] = num_premptions[i] / total_jobs_per_priority[i]

plt.bar(range(len(avg_premptions)), avg_premptions)
plt.xlabel('Priority Level')
plt.ylabel('Average Preemptions')
plt.title('Average Preemptions Vs Priority Level')
plt.xticks(range(len(avg_premptions)))
plt.show()


## Average Latency Vs Priority

In [ ]:
total_jobs_per_priority = [0] * 4
avg_latency_due_to_preemption = [0] * 4

for job in jobsArray:
    total_jobs_per_priority[int(job.priority)] += 1
    avg_latency_due_to_preemption[int(job.priority)] += job.latency_due_to_preemption

for i in range(4):
    if total_jobs_per_priority[i] != 0:
        avg_latency_due_to_preemption[i] /= total_jobs_per_priority[i]

plt.bar(range(len(avg_latency_due_to_preemption)), avg_latency_due_to_preemption)
plt.xlabel('Priority Level')
plt.ylabel('Average Latency')
plt.title('Average Latency Vs Priority Level')
plt.xticks(range(len(avg_latency_due_to_preemption)))
plt.show()
